In [6]:
class App(object):
    # انیشیال و مقدار های اولیه 
    def __init__(self, video_src):
        #  خاندن ویدو و ریختن ان داخل متغیر کم 
        self.cam = video.create_capture(video_src, presets['cube'])
        # خاندن اولین فریم
        _ret, self.frame = self.cam.read()
        # تعریف اسم پنجره 
        cv.namedWindow('camshift')
        # ست کردن یک کالبک موس بر روی پنجره که بالا تعریف شده
        cv.setMouseCallback('camshift', self.onmouse)

        # مقدار دهی های اولیه
        self.selection = None
        self.drag_start = None
        self.show_backproj = False
        self.track_window = None
        
    # تابع برای تعامل با موس و سیو کردن مختصات کلیک شده
    def onmouse(self, event, x, y, flags, param):
        if event == cv.EVENT_LBUTTONDOWN:
            self.drag_start = (x, y)
            self.track_window = None
        if self.drag_start:
            xmin = min(x, self.drag_start[0])
            ymin = min(y, self.drag_start[1])
            xmax = max(x, self.drag_start[0])
            ymax = max(y, self.drag_start[1])
            self.selection = (xmin, ymin, xmax, ymax)
        if event == cv.EVENT_LBUTTONUP:
            self.drag_start = None
            self.track_window = (xmin, ymin, xmax - xmin, ymax - ymin)

    # فانکشن برای نمایش هیستوگرام رنگی
    def show_hist(self):
        bin_count = self.hist.shape[0]
        bin_w = 24
        img = np.zeros((256, bin_count*bin_w, 3), np.uint8)
        for i in xrange(bin_count):
            h = int(self.hist[i])
            cv.rectangle(img, (i*bin_w+2, 255), ((i+1)*bin_w-2, 255-h), (int(180.0*i/bin_count), 255, 255), -1)
        img = cv.cvtColor(img, cv.COLOR_HSV2BGR)
        cv.imshow('hist', img)

    # فانکشن اصلی برنامه که فریم‌ها را می‌گیرد و پردازش می‌کند
    def run(self):
        while True:
            # خواندن فرم ها
            _ret, self.frame = self.cam.read()
            # کپی کردن فرم داخل متغیر ویس
            vis = self.frame.copy()
            # تبدیل فریم از بی جی ار تو اچ اس وی 
            hsv = cv.cvtColor(self.frame, cv.COLOR_BGR2HSV)
            # ایجاد یک ماسک با استفاده از اینرنج برای فیلتر کردن بازه خاص
            mask = cv.inRange(hsv, np.array((0., 60., 32.)), np.array((180., 255., 255.)))

            # اگر کاربر قسمت سلکت کرده باشد
            if self.selection:
                # خاندن مختصات و ایجاد ماسک و حساب کردن ان براساس هیو و نمایشش
                x0, y0, x1, y1 = self.selection
                hsv_roi = hsv[y0:y1, x0:x1]
                mask_roi = mask[y0:y1, x0:x1]
                hist = cv.calcHist( [hsv_roi], [0], mask_roi, [16], [0, 180] )
                cv.normalize(hist, hist, 0, 255, cv.NORM_MINMAX)
                self.hist = hist.reshape(-1)
                self.show_hist()
                # حذف موارد اضافی
                vis_roi = vis[y0:y1, x0:x1]
                cv.bitwise_not(vis_roi, vis_roi)
                vis[mask == 0] = 0

            #  بررسی اینکه آیا قبلاً قسمت برای ردیابی انتخاب شده است و ابعاد آن درسته یا نه
            if self.track_window and self.track_window[2] > 0 and self.track_window[3] > 0:
                #  ست کردن یلکت مقدار نان برای غیرفعال کردن سلکشن
                self.selection = None
                #  backprojection محاسبه 
                prob = cv.calcBackProject([hsv], [0], self.hist, [0, 180], 1)
                # ترکیب پراب با ماسک
                prob &= mask

                # اجرای الگوریتم کم شیفت برای پیدا کردن موقعیت جدید شیع و به‌روزرسانی
                term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
                track_box, self.track_window = cv.CamShift(prob, self.track_window, term_crit)

                # نمایش prob backprojection را فعال کرده
                if self.show_backproj:
                    vis[:] = prob[...,np.newaxis]
                try:
                    cv.ellipse(vis, track_box, (0, 0, 255), 2)
                except:
                    print(track_box)

            # نمایش فریم
            cv.imshow('camshift', vis)

            ch = cv.waitKey(5)
            # اسکیپ برای بریک کردن از حلقه
            if ch == 27:
                break
            #  بی برای نمایش یا عدم نمایش backprojection     
            if ch == ord('b'):
                self.show_backproj = not self.show_backproj
        # بستن تمام پنجره ها        
        cv.destroyAllWindows()
